# Build plausibilty seed files

### Overview
This notebook converts the clinical plausibility definitions from the OHDSI OMOP Data Quality Dashboard into a format for assessing FHIR data quality using the Qualifier reference implementation. 

Qualifier includes the output of this notebook in its seed directory - you only need to re-generate these files to  account for updates to the OMOP concept terminology values or a change in the Data Quality Dashboard files.

### Retrieving the Data Files
Download the Data Quality Dashboard concept plausibility definitions named `OMOP_CDMv5.3.1_Concept_Level.csv` from the github repository at `https://github.com/OHDSI/DataQualityDashboard/tree/main/inst/csv`.

Use the `https://athena.ohdsi.org/vocabulary/list` app to download the OMOP terminology tables (free registration required). Be sure to include the following vocabularies: `SNOMED`, `ICD9CM`, `ICD10CM`, `CPT4`, `LOINC`, `RXNORM`. Unzip the downloaded terminology file into a local directory.

Update the paths to the plausibility file and the terminology tables in the next cell to reflect the directories where you placed the files.


### Data File Paths

In [ ]:
# Update to the paths of the files you retrieved as described above
concept_path = "./omop/CONCEPT.csv"
concept_cpt_4_path = "./omop/CONCEPT_CPT4.csv"
concept_relationship_path = "./omop/CONCEPT_RELATIONSHIP.csv"
concept_expectations_path = "./omop/OMOP_CDMv5.3.1_Concept_Level.csv"

# Update the output path for the seed files
output_obs_plausibility_path = "./seeds/obs_plausibility.csv"
output_gender_plausibility_path = "./seeds/gender_plausibility.csv"

In [ ]:
# Map OMOP concept names to FHIR terminology systems
terminology_systems = [
    ["CPT4", "http://www.ama-assn.org/go/cpt", "urn:oid:2.16.840.1.113883.6.12"],
    ["UCUM", "http://unitsofmeasure.org", "urn:oid:2.16.840.1.113883.6.8"],
    ["LOINC", "http://loinc.org", "urn:oid:2.16.840.1.113883.6.1"],
    ["ICD9CM", "http://hl7.org/fhir/sid/icd-9-cm", "urn:oid:2.16.840.1.113883.6.2"],
    ["ICD10CM", "http://hl7.org/fhir/sid/icd-10-cm", "urn:oid:2.16.840.1.113883.6.90"],
    ["SNOMED", "http://snomed.info/sct", "urn:oid:2.16.840.1.113883.6.96"],
    ["RXNORM", "http://www.nlm.nih.gov/research/umls/rxnorm", "urn:oid:2.16.840.1.113883.6.88"]
]

In [ ]:
# load duckdb to do the terminology expansion
%pip install duckdb

In [ ]:
# Create an in-memory database and ensure it's working correctly
import duckdb
con = duckdb.connect(database=':memory:')
con.execute('SELECT 42').fetchall()

In [ ]:
# import the concept csv
con.execute(f'''
    DROP TABLE IF EXISTS concept;
    
    CREATE TABLE concept (
        concept_id integer NOT NULL,
        concept_name varchar NULL, --made nullable because of AMA requirements
        domain_id varchar(20) NOT NULL,
        vocabulary_id varchar(20) NOT NULL,
        concept_class_id varchar(20) NOT NULL,
        standard_concept varchar(1) NULL,
        concept_code varchar(50) NOT NULL,
        valid_start_date date NOT NULL,
        valid_end_date date NOT NULL,
        invalid_reason varchar(1) NULL 
    );
    
    CREATE INDEX idx_concept_concept_id ON concept  (concept_id ASC);
    CREATE INDEX idx_concept_code ON concept (concept_code ASC);
    CREATE INDEX idx_concept_vocabluary_id ON concept (vocabulary_id ASC);
    CREATE INDEX idx_concept_domain_id ON concept (domain_id ASC);
    CREATE INDEX idx_concept_class_id ON concept (concept_class_id ASC);

    COPY concept FROM '{concept_path}' (DELIMITER E'\t', HEADER, QUOTE E'\b', DATEFORMAT '%Y%m%d');
    
    COPY concept FROM '{concept_cpt_4_path}' (DELIMITER E'\t', HEADER, QUOTE E'\b', DATEFORMAT '%Y%m%d');
    
    SELECT COUNT(*) FROM concept;

''').fetchall()

In [ ]:
# import concept_relationship csv
con.execute(f'''
    DROP TABLE IF EXISTS concept_relationship;
    
    CREATE TABLE concept_relationship (
        concept_id_1 integer NOT NULL,
        concept_id_2 integer NOT NULL,
        relationship_id varchar(20) NOT NULL,
        valid_start_date date NOT NULL,
        valid_end_date date NOT NULL,
        invalid_reason varchar(1) NULL
    );
            
    CREATE INDEX idx_concept_relationship_id_1  ON concept_relationship  (concept_id_1 ASC);
    CREATE INDEX idx_concept_relationship_id_2 ON concept_relationship (concept_id_2 ASC);
    CREATE INDEX idx_concept_relationship_id_3 ON concept_relationship (relationship_id ASC);

    COPY concept_relationship FROM '{concept_relationship_path}' (DELIMITER E'\t', HEADER, QUOTE E'\b', DATEFORMAT '%Y%m%d');
 
    SELECT COUNT(*) FROM concept_relationship;

''').fetchall()

In [ ]:
# import plausibility csv
con.execute(f'''
    DROP TABLE IF EXISTS concept_expectations;
    
    CREATE TABLE concept_expectations AS 
        SELECT * FROM '{concept_expectations_path}';
 
    SELECT COUNT(*) FROM concept_expectations;

''').fetchall()

In [ ]:
# create SQL for mapping between OMOP and FHIR terminology systems with URIs
sql_fragments = map(lambda t: f"WHEN concept.vocabulary_id = '{t[0]}' THEN '{t[1]}'", terminology_systems)
terminology_sql = 'CASE\n' + '\n'.join(list(sql_fragments)) + '\nEND'
print(terminology_sql)

In [ ]:
# create SQL for mapping between OMOP and FHIR terminology systems with OIDs
sql_fragments = map(lambda t: f"WHEN concept.vocabulary_id = '{t[0]}' THEN '{t[2]}'", terminology_systems)
terminology_sql_oid = 'CASE\n' + '\n'.join(list(sql_fragments)) + '\nEND'
print(terminology_sql_oid)

In [ ]:
# Build seed file for quantitative value range expectations
result = con.execute(f'''
WITH base_codes AS (
    SELECT 
        concept.concept_id AS concept_id,
        {terminology_sql} AS system, 
        {terminology_sql_oid} AS system_oid, 
        concept.concept_code AS code,
        unit_concept.concept_code AS ucum_code,
        expectations."conceptName" AS concept_name,
        expectations."plausibleValueLow" AS low,
        expectations."plausibleValueHigh" AS high

    FROM concept_expectations expectations

    INNER JOIN concept
        ON concept.concept_id = expectations."conceptId"
        AND concept.vocabulary_id IN ('CPT4','LOINC','ICD9CM','ICD10CM','SNOMED')

    INNER JOIN concept AS unit_concept
        ON expectations."unitConceptId" IS NOT NULL
        AND unit_concept.concept_id = expectations."unitConceptId"

    WHERE expectations."plausibleValueLow" IS NOT NULL
        OR expectations."plausibleValueHigh" IS NOT NULL
),
expansion_codes AS (
    SELECT 
        concept.concept_id,
        {terminology_sql} AS system, 
        {terminology_sql_oid} AS system_oid, 
        concept.concept_code AS code,
        ucum_code,
        base_codes.concept_name,
        low,
        high

    FROM base_codes

    INNER JOIN CONCEPT_RELATIONSHIP
        ON concept_relationship.concept_id_1 = base_codes.concept_id
        AND concept_relationship.relationship_id LIKE '%eq'
        AND concept_relationship.invalid_reason IS NULL

    INNER JOIN CONCEPT
        ON concept.concept_id = concept_relationship.concept_id_2
        AND concept.vocabulary_id IN ('CPT4','LOINC','ICD9CM','ICD10CM','SNOMED')
        AND concept.invalid_reason IS NULL
),
merged_codes AS (
    SELECT * FROM base_codes
    UNION ALL
    SELECT * FROM expansion_codes
),
final AS (
    SELECT 
        CONCAT(concept_name, ' between ', low, ' and ', high, ' ', ucum_code) as expectation_name,
        system,
        system_oid,
        code,
        ucum_code,
        low,
        high
    FROM merged_codes
    GROUP BY 1,2,3,4,5,6,7
)

SELECT *
FROM final
''').df()

result.to_csv(
    output_obs_plausibility_path, 
    columns=['expectation_name','system','system_oid','code','ucum_code','low','high'],
    index=None
)

with open(output_obs_plausibility_path, 'r') as fp:
    print(len(fp.readlines()), "lines written")

In [ ]:
result = con.execute(f'''
WITH base_codes AS (
    SELECT 
        concept.concept_id AS concept_id,
        {terminology_sql} AS system,
        {terminology_sql_oid} AS system_oid,
        concept.concept_code AS code,
        expectations."conceptName" AS concept_name,
        expectations."plausibleGender" AS plausible_gender,
        CASE 
            WHEN expectations."cdmTableName" = 'CONDITION_OCCURRENCE' THEN 'condition'
            WHEN expectations."cdmTableName" = 'PROCEDURE_OCCURRENCE' THEN 'procedure'
        END AS resource_type

    FROM concept_expectations expectations 

    INNER JOIN CONCEPT
        ON concept.concept_id = expectations."conceptId"
        AND concept.vocabulary_id IN ('CPT4','LOINC','ICD9CM','ICD10CM','SNOMED')

    WHERE expectations."plausibleGender" IS NOT NULL
),
expansion_codes AS (
    SELECT 
        concept.concept_id,
        {terminology_sql} AS system, 
        {terminology_sql_oid} AS system_oid,
        concept.concept_code AS code,
        base_codes.concept_name,
        base_codes.plausible_gender,
        resource_type

    FROM base_codes

    INNER JOIN concept_relationship
        ON concept_relationship.concept_id_1 = base_codes.concept_id
        AND concept_relationship.relationship_id LIKE '%eq'
        AND concept_relationship.invalid_reason IS NULL

    INNER JOIN concept
        ON concept.concept_id = concept_relationship.concept_id_2
        AND concept.vocabulary_id IN ('CPT4','LOINC','ICD9CM','ICD10CM','SNOMED')
        AND concept.invalid_reason IS NULL
),
merged_codes AS (
    SELECT * FROM base_codes
    UNION ALL
    SELECT * FROM expansion_codes
),
final AS (
    SELECT 
        CONCAT(concept_name, ' gender is ', plausible_gender) as expectation_name,
        system,
        system_oid,
        code,
        CASE 
            WHEN plausible_gender = 'Female' THEN 'F'
            WHEN plausible_gender = 'Male' THEN 'M'
            ELSE plausible_gender
        END AS gender,
        resource_type
        
FROM merged_codes
    GROUP BY 1,2,3,4,5,6
)

SELECT *
FROM final
''').df()

result.to_csv(
    output_gender_plausibility_path, 
    columns=['expectation_name','system','system_oid','code','gender','resource_type'],
    index=None
)

with open(output_gender_plausibility_path, 'r') as fp:
    print(len(fp.readlines()), "lines written")